In [9]:
# basic
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) #不要科學記號
pd.set_option('display.float_format',lambda x : '%.4f' % x)
pd.set_option('display.max_rows', 10)

# 獲得股票資訊
import pandas_datareader as pdr
import yfinance as yf

# 繪圖用的套件
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#獲得時間
import datetime as datetime
import time

#talib計算金融技術指標
import talib
from talib import abstract

import requests

today = datetime.datetime.now() 
day_ago = today - datetime.timedelta(days=100)

import pandas as pd
from datetime import date
import pymssql 

In [10]:
# 創建 ms sql伺服器 教學文
# https://dotblogs.com.tw/Mickey/2016/12/04/194101
# https://afdts.blogspot.com/2019/03/operationalerror-20009-bdb-lib-error.html
# https://dotblogs.com.tw/caubekimo/2018/09/17/145733

In [11]:
try:
    conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
    cursor = conn.cursor()
    sql = 'select * from UserList'
    df_user = pd.read_sql(sql, conn)
    print('good')
except Exception as e:
    a = e
    print(e)

good


In [18]:
conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList
sql = 'select * from StrategyList'
df_stg = pd.read_sql(sql, conn)      # 2 StrategyList
sql = 'select * from TodaySignal'
df_signal = pd.read_sql(sql, conn)   # 3 TodaySignal

In [32]:
df_user

,ID,Name,0,1,2
0,u3f83uhd,Emery,0,1,0
1,dsfvsd,Sandy,0,1,1
2,dsfvsd,Sandy,0,1,1
3,fsavsv,Ken,1,1,1


# 連接資料庫

In [16]:
stg_name = df_stg['StrategyName'].tolist()
stg_name = [str(x.strip()) for x in stg_name]  #去除空白!!
stg_par = df_stg['Parameter'].tolist() 
stg_par = [str(x.strip()) for x in stg_par]
stg_index = [str(x) for x in df_stg.index.tolist()]
print(stg_name,stg_par)

AttributeError: 'NoneType' object has no attribute 'strip'

In [30]:
sql = """
          INSERT INTO UserList (ID,Name,[0],[1],[2]) 
          VALUES(%s,%s,%s,%s,%s)
        """
cursor.execute(sql, ('fsavsv', 'Ken', '1', '1', '1'))
conn.commit()

In [31]:
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList
df_user

,ID,Name,0,1,2
0,u3f83uhd,Emery,0,1,0
1,dsfvsd,Sandy,0,1,1
2,dsfvsd,Sandy,0,1,1
3,fsavsv,Ken,1,1,1


# 股票代號

In [21]:
def stk_symbol():
    res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
    df = pd.read_html(res.text)[0]
    df.drop(columns=[0,1,4,5,7,8,9], inplace=True) 
    df.columns = df.iloc[0]
    df.rename(columns = {'有價證券代號':'股票代號','有價證券名稱':'股票名稱'} , inplace = True)
    df = df.iloc[1:]
    df.reset_index(inplace = True)
    df.drop(columns=['index'], inplace=True) 
    return df
df_stk = stk_symbol()
df_stk  #.groupby('產業別').count()

,股票代號,股票名稱,產業別
0,1101,台泥,水泥工業
1,1102,亞泥,水泥工業
2,1103,嘉泥,水泥工業
3,1104,環泥,水泥工業
4,1108,幸福,水泥工業
...,...,...,...
948,9944,新麗,其他業
949,9945,潤泰新,其他業
950,9946,三發地產,建材營造業
951,9955,佳龍,其他業


# 一、跑每日策略

In [ ]:
df = yf.Ticker('2330.TW').history(period='1d', start=day_ago, end=today)

In [ ]:
def StockData(stock_symbol):
    df = yf.Ticker(stock_symbol).history(period='1d', start=day_ago, end=today)
    df.rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume': 'volume', 'Dividends': 'dividends'}, inplace=True)
    df = df.astype('float')

    # 準備一份你想要計算並且併入 df 的技術指標清單
    ta_list = stg_par

    # ta_list = talib.get_functions() 全部指標

    for x in ta_list:
        try:
            # x 為技術指標的代碼，透過迴圈填入，再透過 eval 計算出 output
            output = eval('abstract.'+x)
            # 如果輸出是一維資料，幫這個指標取名為 x 本身；多維資料則不需命名
            output.name = x.lower() if type(output) == pd.core.series.Series else None
            # 透過 merge 把輸出結果併入 df DataFrame
            df = pd.merge(df, pd.DataFrame(output), left_on = df.index, right_on = output.index)
            df = df.set_index('key_0')
        except:
            print('Error:', x)
    df.reset_index(inplace = True)
    return df

In [ ]:
df_index = StockData('2330.TW')
df_index

## 每日訊號

In [ ]:
def signal():
    signal_list = []
    for i in range( len(df_stk) ):
        one_list = []
        try:
            df_index = StockData(df_stk['股票代號'].loc[i]+'.TW')
            today = df_index.loc[len(df_index)-1]
            yesterday = df_index.loc[len(df_index)-2]
            print('s')
            # 布林通道策略:今日股價由下而上 穿過下緣
            if ( yesterday['lowerband'] > yesterday['close'] ) and ( today['lowerband'] < today['close'] ):
                print(df_stk['股票代號'].loc[i], '布林:收盤價由下而上，穿過下緣')
                one_list.append(1)
            else:
                one_list.append(0)
                
            # RSI策略:
            if ( yesterday['rsi(df, timeperiod=14)'] < 30 ):
                print(df_stk['股票代號'].loc[i], 'RSI:小於30，處於超賣區')
                one_list.append(1)
            else:
                one_list.append(0)   
                
            # MACD策略:
            if ( yesterday['macd'] < yesterday['macdsignal'] ) and ( today['macd'] > today['macdsignal'] ):
                print(df_stk['股票代號'].loc[i], 'MACD:快線向上突破慢線')
                one_list.append(1)
            else:
                one_list.append(0)
            signal_list.append(one_list)
        except:
            signal_list.append([0,0,0])
            print(df_stk['股票代號'].loc[i], 'error')
    return signal_list

In [ ]:
a = signal()

s
2414 error
s
2415 error
s
2417 error
s
2419 error
s
2420 error
s
2421 error
s
2423 error
s
2424 error
s
2425 error
s
2426 error
s
2427 error
s
2428 error
s
2429 error
s
2430 error
s
2431 error
s
2433 error
s
2434 error
s
2436 error
s
2438 error
s
2439 error
s
2440 error
s
2441 error
s
2442 error
s
2443 error
s
2444 error
s
2449 error
s
2450 error
s
2451 error
s
2453 error
s
2454 error
s
2455 error
s
2456 error
s
2457 error
s
2458 error
s
2459 error
s
2460 error
s
2461 error
s
2462 error
s
2464 error
s
2465 error
s
2466 error
s
2467 error
s
2468 error
s
2471 error
s
2472 error
s
2474 error
s
2476 error
s
2477 error
s
2478 error
s
2480 error
s
2481 error
s
2482 error
s
2483 error
s
2484 error
s
2485 error
s
2486 error
s
2488 error
s
2489 error
s
2491 error
s
2492 error
s
2493 error
s
2495 error
s
2496 error
s
2497 error
s
2498 error
s
2501 error
s
2504 error
s
2505 error
s
2506 error
s
2509 error
s
2511 error
s
2514 error
s
2515 error
s
2516 error
s
2520 error
s
2524 error
s
2527 error

In [ ]:
df_a = pd.DataFrame(a, columns = stg_name)
df_all_index = pd.concat([df_stk, df_a], axis=1)
df_all_index

## 每日訊號-寫入資料庫

In [ ]:
#清空資料表
conn = pymssql.connect(server='104.199.210.142', user='sqlserver', password='sqlserver', database='LineBot')
cursor = conn.cursor()
sql = """
      TRUNCATE TABLE 當日股票訊號;
    """
cursor.execute( sql )
conn.commit()
print('END')

In [ ]:
for index, row in df_all_index.iterrows():
    try:
        sql = """
          INSERT INTO 當日股票訊號 (股票代號,股票名稱,產業別,[0],[1],[2]) 
          VALUES(%s,%s,%s,%s,%s,%s)
        """
        cursor.execute(sql, (row.股票代號, row.股票名稱, row.產業別, row.BBAND, row.RSI, row.MACD))
        conn.commit()

    except:
        print('Error:', row.股票代號, ' ', row.股票名稱, ' ', row.產業別)

cursor.close()

In [ ]:
# 檢查是否寫入
conn = pymssql.connect(server='104.199.210.142', user='sqlserver', password='sqlserver', database='LineBot')
cursor = conn.cursor()
sql = 'select * from 當日股票訊號'
df_signal = pd.read_sql(sql, conn)
df_signal